In [1]:
!nvidia-smi

Thu Dec 15 12:31:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.11    Driver Version: 525.60.11    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   23C    P8     7W / 250W |    344MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:17:00.0 Off |                  N/A |
| 33%   

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.13.1+cu116 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', major=8, minor=6, total_memory=24258MB, multi_processor_count=82)


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [3]:
# setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import glob
import pandas as pd

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.structures import BoxMode, Instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import math
import torch
from skimage.draw import ellipse, ellipse_perimeter

In [ ]:
import os
import random
import numpy as np
seed = 5

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("done")
seed_everything(seed)


# Train YOLOv5s on the dataset 
!python train.py --img 640 --batch 16 --epochs 100 --data parasite.yaml --weights yolov5s.pt --nosave --cache

done
train: weights=yolov5s.pt, cfg=, data=parasite.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=16, img_size=[640], rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=True, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-290-g62409ee torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0, hsv_s=0, hsv_v=0, degrees=0, translate=0, scale=0, shear=0.0, perspective=

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Defined a function to create a dictionary in a format accepted by detectron2
def get_cysts_dicts(img_dir):
    # load the JSON file
    json_file = os.path.join(img_dir, "via_final.json")
    with open(json_file) as f:
        imgs_anno = json.load(f)
    #list_dict =[]
    #for i, v in enumerate(imgs_anno.values()):
      #list_dict.append(v)
    #imgs_anns = list_dict[1]
    dataset_dicts = []
    
    # loop through the entries in the JSON file
    for idx, v in enumerate(imgs_anno.values()):
        record = {}  # Initialize dict to add image name, image id and annotation in a format accepted by detectron2
        
        filename = os.path.join(img_dir, v["filename"])
        
        if os.path.isfile(filename):
          pass
        else:
          print("Image not found", filename)
          continue
        
        # height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        # record["image_id"] = idx
        # record["height"] = height
        # record["width"] = width
        annos = v["regions"]

        objs = []
        # one image can have multiple annotations, therefore this loop is needed
        for annotation in annos:
            # reformat the polygon information to fit the specifications
            anno = annotation["shape_attributes"]
            cx = anno["cx"]
            cy = anno["cy"]
            rx= anno["rx"]
            ry=anno["ry"]
            theta =anno["theta"]
            region_attributes = annotation["region_attributes"]["cysts"]
            xx,yy = ellipse_perimeter(cx, cy, round(rx), round(ry), orientation= theta)
            poly = [(x+0.5, y + 0.5) for x,y in zip(xx,yy)]
            poly = [p for x in poly for p in x]
            # specify the category_id to match with the class.

            if "Giardia" in region_attributes:
                category_id = 1
            elif "Crypto" in region_attributes:
                category_id = 0
            
            # def ellipse_to_rectange(rx,ry,theta):
            #   ea = rx*math.cos(theta)
            #   eb = ry*math.sin(theta)
            #   ec = rx*math.sin(theta)
            #   ed = ry*math.cos(theta)
            #   width = math.sqrt(math.pow(ea,2)+math.pow(eb,2))*2
            #   height = math.sqrt(math.pow(ec,2)+math.pow(ed,2))*2
            #   return (width ,height)
            # width , height = ellipse_to_rectange(rx,ry,theta)

            obj = {
                # "bbox": [cx-width*0.5,cy-height*0.5,cx+width*0.5,cy+height*0.5],
                # "bbox_mode": BoxMode.XYXY_ABS,
                #"segmentation": [poly],
                "segmentation": [],
                "category_id": category_id,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
     
    return dataset_dicts

In [ ]:
!ls

In [ ]:
# dataset_val = get_cysts_dicts("/content/drive/My Drive/cysts_dataset_4/Test_set_sample_new")
dataset_val = get_cysts_dicts("/content/drive/My Drive/smartphone/selected_test_images")

In [ ]:
len(dataset_val)

193

In [ ]:
count_giardia =[]
count_crypto =[]
for r in dataset_val:
  image = r
  anno = r['annotations']   #all annotations of ground truth
  for a in range(len(anno)):
    if image['annotations'][a]['category_id'] == 0:
      count_crypto.append(0)
    else:
      count_giardia.append(1)
print(f"total_annotation_Giardia: {len(count_giardia)}")
print(f"total_annotation_Crypto: {len(count_crypto)}")
total_anno_g = len(count_giardia)
total_anno_c =len(count_crypto)

total_annotation_Giardia: 166
total_annotation_Crypto: 137


In [ ]:
# Detect the parasites from the images
!python detect.py --weights '/content/drive/My Drive/new_outputs/last_5.pt' --img 1280 --conf 0.3 --save-txt --source "/content/drive/My Drive/smartphone/selected_test_1280/images" 

In [ ]:
!ls

CONTRIBUTING.md  export.py   new_outputs       train.py
data		 hubconf.py  README.md	       tutorial.ipynb
detect.py	 LICENSE     requirements.txt  utils
Dockerfile	 models      runs	       val.py


In [ ]:
%cd runs/detect/exp3

/content/yolov5/runs/detect/exp3


## TP, FP, FN calculator

In [ ]:
df=pd.DataFrame(columns=["image_name","TP_g","FP_g","TP_c","FP_c"])

In [ ]:
# Import necessary libraries
from detectron2.structures import Boxes
from detectron2.structures import pairwise_iou
import numpy as np
import torch
g_count=0

In [ ]:
# Method to convert labels txt to boxes
def convert_labels_to_boxes(file, img_height, img_width):
  
  read_file = file.read().split('\n')      # Keep lines the file as array element
  # print(read_file)
  read_file = read_file[0:-1]           # Revome end element (i.e ' ')
  bboxs= []
  category_id = []
  
  for i in range(len(read_file)):  # Loop through each line of the file
    b= read_file[i].split(' ')   # Separate each elements of the line
    
    category = int(b[0])
    nor_x_center = float(b[1])
    nor_y_center = float(b[2])
    nor_w = float(b[3])
    nor_h = float(b[4])

    # Multiply with actual height and width to un normalize
    act_x_center = nor_x_center * img_width
    act_y_center = nor_y_center * img_height
    act_w = nor_w * img_width
    act_h = nor_h * img_height
    
    # Conversion from Yolo style to Pascal_voc (act is actual)
    x_min = act_x_center - (act_w / 2)
    y_min = act_y_center - (act_h / 2)
    x_max = act_x_center + (act_w / 2)
    y_max = act_y_center + (act_h / 2)

    list_box = [x_min, y_min, x_max, y_max]
    bboxs.append(list_box)
    category_id.append(category)
  
  bboxs = np.array(bboxs)
  b_torch = torch.from_numpy(bboxs)
  b_boxes = Boxes(b_torch)
  file.close()
  return b_boxes, category_id

In [ ]:
def calculate_parameters(iou_output, gt_class, pred_class):
  TP_g = 0
  FP_g = 0
  TP_c =0
  FP_c =0
  p_count = 0

  for i in iou_output.numpy():
    
    if not np.any(i):
      if pred_class[p_count] == 0:
          FP_c +=1
      elif pred_class[p_count] == 1:
          FP_g +=1
      p_count +=1     
      continue 
    i_list= i.tolist()  # converting numpy array to list to get the max value and index
    max_value = max(i_list)  
    index = i_list.index(max_value)
    if pred_class[p_count] == gt_class[index] and max_value >0.3:
        if pred_class[p_count] == 0:
            TP_c +=1
        else:
            TP_g +=1
    else:
        if pred_class[p_count]== 0:
            FP_c +=1
        else:
            FP_g +=1 
    p_count +=1
  print(f"TP_g : {TP_g}")
  print(f"FP_g: {FP_g}")  
  print(f"TP_c : {TP_c}")  
  print(f"FP_c : {FP_c}") 
  return TP_g, TP_c, FP_g, FP_c 

In [ ]:
a="/content/drive/My Drive/selected_test_1280/images/a.jpeg"
a[50:]

'a.jpeg'

In [ ]:
import cv2
true_giardia=[] 
true_crypto= [] 
false_giardia= [] 
false_crypto=[]
output_image = []

for i in range(205): # Loop through all the labels
  # For ground truth
  path_gt=f"/content/drive/My Drive/smartphone/selected_test_1280/labels/{i}.txt"
  img_path= f"/content/drive/My Drive/smartphone/selected_test_1280/images/{i}.jpeg"
  image = cv2.imread(img_path)
  try:
    img_height, img_width, _ = image.shape
    output_image.append(img_path[50:])
  # Try and except for no file case
  except:
    continue
  try:
    file=open(path_gt, "r")
    b_boxes, category_id = convert_labels_to_boxes(file, img_height, img_width)
  
  except:
    bboxs = np.array([])
    b_torch = torch.from_numpy(bboxs)
    b_boxes = Boxes(b_torch)
    category_id=[]
    
  
  # print(path_gt)
  # print(b_boxes,category_id)
  # print('\n')

  path_pred=f"runs/detect/exp7/labels/{i}.txt"
  
  try:
    file_pred = open(path_pred, "r")
    pred_box, pred_class = convert_labels_to_boxes(file_pred,img_height, img_width)
  
  except:
    bboxs = np.array([])
    b_torch = torch.from_numpy(bboxs)
    pred_box= Boxes(b_torch)
    pred_class=[]
    pass
  
  iou_output = pairwise_iou(pred_box, b_boxes)
  print({path_gt})
  print(path_pred)
  print(f"Ground truth: {b_boxes}")
  print(f"Pred box: {pred_box}")
  print(iou_output)
  TP_g, TP_c , FP_g, FP_c =calculate_parameters(iou_output, category_id, pred_class)

# Append parasites of all the images
  true_giardia.append(TP_g)
  true_crypto.append(TP_c)
  false_giardia.append(FP_g)
  false_crypto.append(FP_c)
  print('\n')

# Convert the list into array to sum it as a whole
np_true_giardia = np.array(true_giardia)
np_true_crypto = np.array(true_crypto)
np_false_giardia = np.array(false_giardia)
np_false_crypto = np.array(false_crypto)

# Print the sum of TP, FP of Giardia and Cryptosporidium 
print(f"true_giardia: {np_true_giardia.sum()}")
print(f"true_crypto: {np_true_crypto.sum()}")
print(f"false_giardia: {np_false_giardia.sum()}")
print(f"false_crypto: {np_false_crypto.sum()}")

TP_giardia = np_true_giardia.sum()
TP_crypto = np_true_crypto.sum()
FP_giardia = np_false_giardia.sum()
FP_crypto = np_false_crypto.sum()

{'/content/drive/My Drive/smartphone/selected_test_1280/labels/0.txt'}
runs/detect/exp7/labels/0.txt
Ground truth: Boxes(tensor([[574.2635, 365.5814, 587.9070, 379.2248],
        [348.8372, 351.0078, 366.8217, 368.9922]]))
Pred box: Boxes(tensor([[569.9996, 367.0006, 585.9997, 381.0006]]))
tensor([[0.5380, 0.0000]])
TP_g : 0
FP_g: 0
TP_c : 1
FP_c : 0


{'/content/drive/My Drive/smartphone/selected_test_1280/labels/1.txt'}
runs/detect/exp7/labels/1.txt
Ground truth: Boxes(tensor([], size=(0, 4)))
Pred box: Boxes(tensor([], size=(0, 4)))
tensor([], size=(0, 0))
TP_g : 0
FP_g: 0
TP_c : 0
FP_c : 0


{'/content/drive/My Drive/smartphone/selected_test_1280/labels/3.txt'}
runs/detect/exp7/labels/3.txt
Ground truth: Boxes(tensor([[375.1938, 371.7830, 401.8605, 395.3488]]))
Pred box: Boxes(tensor([[376.0005, 371.0005, 401.0005, 400.0005]]))
tensor([[0.7709]])
TP_g : 1
FP_g: 0
TP_c : 0
FP_c : 0


{'/content/drive/My Drive/smartphone/selected_test_1280/labels/4.txt'}
runs/detect/exp7/labels/4.txt

In [ ]:
dict_store = {"image_name" : output_image,"TP_g":true_giardia,"FP_g":false_giardia,"TP_c":true_crypto,"FP_c":false_crypto}
df = pd.DataFrame(dict_store)
df.to_excel(f"/content/drive/My Drive/yolo_5.xlsx",index=False) 

## Metrics calculator

In [ ]:
total_anno_c

137

In [ ]:
FN_giardia = total_anno_g - TP_giardia
FN_crypto = total_anno_c - TP_crypto
precision_g = round(TP_giardia/(TP_giardia + FP_giardia),3)
precision_c = round(TP_crypto/(TP_crypto + FP_crypto),3)
recall_g = round(TP_giardia/(TP_giardia + FN_giardia),3)
recall_c = round(TP_crypto/(TP_crypto + FN_crypto),3)
f1_g = round(((2*precision_g * recall_g)/(precision_g + recall_g)),3)
f1_c = round(((2*precision_c * recall_c)/(precision_c + recall_c)),3)
print(f"gp: {precision_g}")
print(f"gr: {recall_g}")
print(f"gf1: {f1_g}\n")
print(f"cp: {precision_c}")
print(f"cr: {recall_c}")
print(f"cf1: {f1_c}")


gp: 0.683
gr: 0.259
gf1: 0.376

cp: 0.42
cr: 0.248
cf1: 0.312


In [ ]:
print(FN_giardia, FN_crypto)

11 10


In [ ]:
path_pred="runs/detect/exp/labels/0.txt"
file=open(path_pred, "r")

In [ ]:
file.read()

'1 0.523438 0.583333 0.0375 0.05\n1 0.549219 0.579167 0.0453125 0.05\n1 0.557813 0.559375 0.046875 0.0520833\n1 0.492969 0.571875 0.0328125 0.04375\n1 0.571875 0.645833 0.034375 0.0416667\n1 0.535156 0.620833 0.0328125 0.0333333\n1 0.58125 0.604167 0.0375 0.0583333\n1 0.514844 0.509375 0.0515625 0.04375\n1 0.682813 0.719792 0.028125 0.05625\n'

In [ ]:
file.read()

'1 0.6828125 0.7229166666666667 0.032786566908721684 0.05600174499804922\n0 0.6203125 0.5208333333333334 0.02124999979868658 0.02666666695186072\n'